## Chatbots with LangGraph

In [ ]:
%pip install langgraph langsmith

In [ ]:
%pip install langchain langchain_community langchain-ollama

In [ ]:
%pip install python-dotenv

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

In [8]:
os.environ["LANGSMITH_API_KEY"] = langsmith_api_key
os.environ["LANGSMITH_PROJECT"] = langsmith_project
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [9]:
from langchain_ollama import ChatOllama

In [10]:
llm = ChatOllama(model="llama3.2")

In [12]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [13]:
class State(TypedDict):
    """
    Messages have the type 'list'. The `add_messages` function
    in the annotation defines how this state key should be updated
    (in this case, it appends messages to the list, rather than overwriting them)
    """
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [14]:
def chatbot(state: State):
    return {"messages": llm.invoke(state['messages'])}

In [15]:
graph_builder.add_node("chatbot", chatbot)

In [16]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [19]:
graph = graph_builder.compile()

In [20]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["q", "quit"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        print(event.values())
        for value in event.values():
            print(value['messages'])
            print("Assistant: ", value['messages'].content)


dict_values([{'messages': AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-08T20:44:13.722878838Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4883724627, 'load_duration': 2587994126, 'prompt_eval_count': 26, 'prompt_eval_duration': 1532000000, 'eval_count': 10, 'eval_duration': 759000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-52a1fc41-3faa-4183-a3b8-84282e8ef707-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36})}])
content='Hello! How can I assist you today?' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-03-08T20:44:13.722878838Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4883724627, 'load_duration': 2587994126, 'prompt_eval_count': 26, 'prompt_eval_duration': 1532000000, 'eval_count': 10, 'eval_duration': 759000000, 'message': Message(role='assistant', 